# gridtracer Data Processing Pipeline Evaluation Notebook

## Overview

This notebook aims to provide an evaluation and analysis of the gridtracer data processing pipeline outputs. It corresponds to the 5 major processing steps in `main.py` and serves two primary purposes:

1. Validate that each pipeline step produces expected outputs with reasonable data quality
2. Generate comprehensive statistics and visualizations for pipeline runs

## Pipeline Steps Evaluated

Based on `run_pipeline_v2()` in `main.py`, this notebook evaluates:

1. **Census Data Analysis** (Step 1) - Regional boundaries, blocks, demographics
2. **NREL Data Analysis** (Step 2) - Building characteristics and vintage distributions  
3. **OSM Data Analysis** (Step 3) - OpenStreetMap features and infrastructure
4. **Building Data Analysis** (Steps 3.5 & 4) - Microsoft Buildings + Classification Heuristics
5. **Road Network Analysis** (Step 5) - Routable network generation and topology

In [ ]:
# Import required libraries
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np
from shapely.geometry import Point, Polygon
import warnings
import geopandas as gpd
import contextily as ctx
from shapely.geometry import box
import random
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

# Configuration - Update these paths based on your output directory structure
BASE_OUTPUT_PATH = Path("/Users/magic-rabbit/Documents/00_Tech-Repositories/05_MASTER_THESIS/gridtracer/gridtracer/data/output")  # Update this path
REGION_PATH = BASE_OUTPUT_PATH / "MA" / "Middlesex_County" / "Cambridge_city"  # Update based on your region

print("gridtracer Pipeline Evaluation Notebook Initialized")
print(f"Base output path: {BASE_OUTPUT_PATH}")
print(f"Region path: {REGION_PATH}")

## 1. Census Data Analysis

This section evaluates the census data processing from Step 1 of the pipeline, which handles:
- Regional boundary definition
- Census block extraction
- Demographic data exploration


In [ ]:
def load_and_validate_census_data(region_path: Path) -> dict:
    """
    Load and validate census data outputs.
    
    Returns:
        dict: Dictionary containing loaded census data and validation results
    """
    census_path = region_path / "CENSUS"
    results = {}
    
    try:
        # Resolve the first match from the generator
        blocks_file = next(census_path.glob("target_region_blocks.geojson"))
        boundary_file = next(census_path.glob("target_region_boundary.geojson"))

        
        results['blocks'] = gpd.read_file(blocks_file)
        results['boundary'] = gpd.read_file(boundary_file)
        results['blocks_file'] = blocks_file
        results['boundary_file'] = boundary_file
        
        print(f"✓ Census data loaded successfully")
        print(f"  - Blocks file: {blocks_file.name}")
        print(f"  - Boundary file: {boundary_file.name}")
        
    except Exception as e:
        print(f"✗ Error loading census data: {e}")
        return {}
    
    return results

def analyze_census_data(census_data: dict) -> None:
    """Perform comprehensive analysis of census data."""
    
    if not census_data:
        print("No census data to analyze")
        return
    
    blocks = census_data['blocks']
    boundary = census_data['boundary']
    
    print("=" * 60)
    print("CENSUS DATA ANALYSIS")
    print("=" * 60)
    
    # Geographic reprojection:

    # Basic statistics

    # EPSG:5070 designed for continental US
    target_crs = "EPSG:5070"
    blocks_projected = blocks.to_crs(target_crs)
    boundary_projected = boundary.to_crs(target_crs)
    
    print(f"\n📊 BASIC STATISTICS:")
    print(f"  -  Census data is reprojected to: {target_crs} (CONUS Albers)")

    print(f"  - Number of census blocks: {len(blocks):,}")
    print(f"  - Total boundary area: {boundary_projected.geometry.area.sum()/1e6:.2f} km²")
    print(f"  - Average block area: {blocks_projected.geometry.area.mean()/1e6:.4f} km²")
    
    # Demographic statistics (if available)
    demographic_cols = [col for col in blocks.columns if any(x in col for x in ['POP20', 'HOUSING20'])]
    if demographic_cols:
        for col in demographic_cols:        
            total = blocks[col].sum()
            print(f"  - Total {col}: {total:,}")
    
    # Sanity checks
    print(f"\n🔍 SANITY CHECKS:")
    
    # Check for empty geometries
    empty_blocks = blocks.geometry.is_empty.sum()
    print(f"  - Empty geometries: {'✓' if empty_blocks == 0 else '✗'} ({empty_blocks} empty)")
    
    # Check spatial coverage
    blocks_within_boundary = blocks.geometry.within(boundary.geometry.iloc[0]).sum()
    coverage_pct = (blocks_within_boundary / len(blocks)) * 100
    print(f"  - Blocks within boundary: {'✓' if coverage_pct > 95 else '✗'} ({coverage_pct:.1f}%)")
    
   # Visualization
    _, ax = plt.subplots(figsize=(8, 6))

    # Map visualization
    boundary.plot(ax=ax, color='red', alpha=0.3, edgecolor='red', linewidth=2)
    blocks.plot(ax=ax, alpha=0.7, edgecolor='black', linewidth=0.5)

    ax.set_title('Census Blocks and Regional Boundary')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    plt.tight_layout()
    plt.show()
    # Execute census data analysis
census_data = load_and_validate_census_data(REGION_PATH)
analyze_census_data(census_data)

In [ ]:
# Analysis on the coverage of the census blocks and the county as a subdivision, as not perfect overlap can be guaranteed.
def analyze_spatial_coverage(blocks_projected: gpd.GeoDataFrame, boundary_projected: gpd.GeoDataFrame) -> dict:
    """
    Analyze and visualize spatial coverage of census blocks within boundary.
    
    This function performs comprehensive spatial coverage analysis including:
    - Classification of blocks as inside, partial, or outside boundary
    - Distance analysis for outside blocks
    - Demographic impact assessment
    - Detailed visualizations with problematic areas highlighted
    
    Args:
        blocks_projected: GeoDataFrame of census blocks in projected CRS
        boundary_projected: GeoDataFrame of boundary in projected CRS
        
    Returns:
        dict: Dictionary containing coverage statistics and analysis results
    """
    print(f"\n🗺️ SPATIAL COVERAGE ANALYSIS:")
    
    results = {
        'within_count': 0,
        'intersect_count': 0,
        'outside_count': 0,
        'partial_count': 0,
        'coverage_within_pct': 0.0,
        'coverage_intersect_pct': 0.0,
        'outside_blocks': None,
        'success': False
    }
    
    try:
        # Use projected coordinates for accurate spatial operations
        blocks_within = blocks_projected.geometry.within(boundary_projected.geometry.iloc[0])
        blocks_intersect = blocks_projected.geometry.intersects(boundary_projected.geometry.iloc[0])
        
        within_count = blocks_within.sum()
        intersect_count = blocks_intersect.sum()
        outside_count = len(blocks_projected) - intersect_count
        
        coverage_within_pct = (within_count / len(blocks_projected)) * 100
        coverage_intersect_pct = (intersect_count / len(blocks_projected)) * 100
        
        # Update results dictionary
        results.update({
            'within_count': within_count,
            'intersect_count': intersect_count,
            'outside_count': outside_count,
            'coverage_within_pct': coverage_within_pct,
            'coverage_intersect_pct': coverage_intersect_pct
        })
        print(f"  - Total blocks: {len(blocks_projected):,}")
        print(f"  - Blocks completely within boundary: {within_count:,} ({coverage_within_pct:.1f}%)")
        print(f"  - Blocks intersecting boundary: {intersect_count:,} ({coverage_intersect_pct:.1f}%)")
        print(f"  - Blocks completely outside: {outside_count:,} ({(outside_count/len(blocks_projected)*100):.1f}%)")
        
        # Identify different categories of blocks
        blocks_inside = blocks_projected[blocks_within]
        blocks_partial = blocks_projected[blocks_intersect & ~blocks_within]  # Intersect but not within
        blocks_outside = blocks_projected[~blocks_intersect]  # Completely outside
        
        partial_count = len(blocks_partial)
        results['partial_count'] = partial_count
        results['outside_blocks'] = blocks_outside
        
        print(f"  - Blocks partially overlapping: {partial_count:,}")
        
        # Analyze the outside blocks
        if len(blocks_outside) > 0:
            print(f"\n🚨 BLOCKS OUTSIDE BOUNDARY ANALYSIS:")
            
            # Calculate distances from boundary for outside blocks
            boundary_geom = boundary_projected.geometry.iloc[0]
            distances = blocks_outside.geometry.centroid.distance(boundary_geom)
            
            print(f"  - Distance range: {distances.min():.0f} - {distances.max():.0f} meters")
            print(f"  - Mean distance: {distances.mean():.0f} meters")
            print(f"  - Median distance: {distances.median():.0f} meters")
            
            # Check if they have population/housing data
            if 'POP20' in blocks_outside.columns:
                outside_pop = blocks_outside['POP20'].sum()
                outside_housing = blocks_outside['HOUSING20'].sum() if 'HOUSING20' in blocks_outside.columns else 0
                print(f"  - Population in outside blocks: {outside_pop:,}")
                print(f"  - Housing units in outside blocks: {outside_housing:,}")
                
                # Store in results
                results['outside_population'] = outside_pop
                results['outside_housing'] = outside_housing
        
        # Create detailed visualization
        fig, axes = plt.subplots(1, 2, figsize=(20, 8))
        
        # Map 1: Overview with all blocks
        boundary_projected.plot(ax=axes[0], color='red', alpha=0.3, edgecolor='red', linewidth=2, label='Boundary')
        
        if len(blocks_inside) > 0:
            blocks_inside.plot(ax=axes[0], color='green', alpha=0.7, edgecolor='darkgreen', 
                              linewidth=0.1, label=f'Inside ({len(blocks_inside):,})')
        
        if len(blocks_partial) > 0:
            blocks_partial.plot(ax=axes[0], color='orange', alpha=0.7, edgecolor='darkorange', 
                               linewidth=0.1, label=f'Partial ({len(blocks_partial):,})')
        
        if len(blocks_outside) > 0:
            blocks_outside.plot(ax=axes[0], color='red', alpha=0.7, edgecolor='darkred', 
                               linewidth=0.1, label=f'Outside ({len(blocks_outside):,})')
        
        axes[0].set_title('Census Blocks Spatial Coverage')
        axes[0].legend()
        axes[0].set_xlabel('Easting (m)')
        axes[0].set_ylabel('Northing (m)')
        
        # Map 2: Focus on problematic areas (if any)
        if len(blocks_outside) > 0 or len(blocks_partial) > 0:
            # Calculate bounds that include boundary and problematic blocks
            all_problematic = pd.concat([blocks_partial, blocks_outside]) if len(blocks_partial) > 0 and len(blocks_outside) > 0 else (blocks_partial if len(blocks_partial) > 0 else blocks_outside)
            
            boundary_projected.plot(ax=axes[1], color='red', alpha=0.3, edgecolor='red', linewidth=2, label='Boundary')
            
            if len(blocks_partial) > 0:
                blocks_partial.plot(ax=axes[1], color='orange', alpha=0.8, edgecolor='darkorange', 
                                   linewidth=0.5, label=f'Partial overlap ({len(blocks_partial):,})')
            
            if len(blocks_outside) > 0:
                blocks_outside.plot(ax=axes[1], color='red', alpha=0.8, edgecolor='darkred', 
                                   linewidth=0.5, label=f'Outside ({len(blocks_outside):,})')
            
            # Add some nearby inside blocks for context
            if len(blocks_inside) > 0:
                sample_inside = blocks_inside.sample(min(100, len(blocks_inside)))
                sample_inside.plot(ax=axes[1], color='lightgreen', alpha=0.6, edgecolor='green', 
                                  linewidth=0.1, label='Inside (sample)')
            
            axes[1].set_title('Focus: Problematic Blocks')
            axes[1].legend()
            axes[1].set_xlabel('Easting (m)')
            axes[1].set_ylabel('Northing (m)')
            
            # Set bounds to focus on problematic areas
            bounds = all_problematic.total_bounds
            buffer = max(bounds[2] - bounds[0], bounds[3] - bounds[1]) * 0.1  # 10% buffer
            axes[1].set_xlim(bounds[0] - buffer, bounds[2] + buffer)
            axes[1].set_ylim(bounds[1] - buffer, bounds[3] + buffer)
        else:
            axes[1].text(0.5, 0.5, 'No problematic blocks found!\nAll blocks are within boundary.', 
                        transform=axes[1].transAxes, ha='center', va='center', fontsize=14,
                        bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgreen", alpha=0.8))
            axes[1].set_title('Spatial Coverage: EXCELLENT')
        
        plt.tight_layout()
        plt.show()
        
        # Additional analysis for problematic blocks
        if len(blocks_outside) > 0:
            print(f"\n🔍 DETAILED ANALYSIS OF OUTSIDE BLOCKS:")
            
            # Show some example outside blocks
            print(f"Sample of blocks outside boundary (first 5):")
            sample_outside = blocks_outside.head()
            
            boundary_geom = boundary_projected.geometry.iloc[0]
            
            for idx, row in sample_outside.iterrows():
                geoid = row.get('GEOID20', idx)
                pop = row.get('POP20', 'N/A')
                housing = row.get('HOUSING20', 'N/A')
                area = row.geometry.area / 1e6  # km²
                
                # Distance from boundary
                dist = row.geometry.centroid.distance(boundary_geom)
                
                print(f"  - Block {geoid}: Pop={pop}, Housing={housing}, Area={area:.4f}km², Distance={dist:.0f}m")
        
        # Recommendations
        print(f"\n💡 SPATIAL COVERAGE RECOMMENDATIONS:")
        
        if coverage_intersect_pct >= 99:
            print(f"  ✅ Excellent spatial coverage - minor edge effects only")
            results['recommendation'] = 'excellent'
        elif coverage_intersect_pct >= 95:
            print(f"  ⚠️ Good coverage - some blocks extend beyond boundary")
            print(f"     Consider: boundary definition or block selection criteria")
            results['recommendation'] = 'good'
        else:
            print(f"  ❌ Poor coverage - significant spatial mismatch")
            print(f"     Action needed: check boundary definition and block selection")
            results['recommendation'] = 'poor'
            
        if len(blocks_outside) > 0:
            outside_with_pop = blocks_outside[blocks_outside.get('POP20', 0) > 0] if 'POP20' in blocks_outside.columns else []
            if len(outside_with_pop) > 0:
                print(f"  ⚠️ {len(outside_with_pop)} outside blocks have population - data may be lost")
                results['data_loss_risk'] = len(outside_with_pop)
        
        results['success'] = True
        
    except Exception as e:
        print(f"  ❌ Error in spatial coverage analysis: {e}")
        results['error'] = str(e)
    
    return results

# In your analyze_census_data function, replace the spatial coverage code with:
blocks_projected = census_data['blocks'].to_crs(epsg=5070)
boundary_projected = census_data['boundary'].to_crs(epsg=5070)
coverage_results = analyze_spatial_coverage(blocks_projected, boundary_projected)

# You can also use the results for further analysis:
if coverage_results['success']:
    print(f"Coverage analysis completed with {coverage_results['coverage_intersect_pct']:.1f}% intersection")
else:
    print(f"Coverage analysis failed: {coverage_results.get('error', 'Unknown error')}")

In [ ]:
# Visualize sample block of your choice

def plot_block_with_population_housing_and_buildings(
    blocks_gdf: gpd.GeoDataFrame,
    target_geoid20: str,
    title: str = "Census Block with Population/Housing Data"
) -> None:
    """
    Plots a specific census block, its population and housing data,
    and an OpenStreetMap basemap which includes building outlines.

    Args:
        blocks_gdf (gpd.GeoDataFrame): GeoDataFrame containing census blocks.
                                       Must include 'GEOID20', 'POP20',
                                       and 'HOUSING20' columns.
        target_geoid20 (str): The GEOID20 of the block to plot.
        title (str, optional): The title for the plot.
                               Defaults to "Census Block with Population/Housing Data".
    """
    # Ensure GEOID20 column is string type for matching
    if 'GEOID20' not in blocks_gdf.columns:
        print("Error: 'GEOID20' column not found in GeoDataFrame.")
        return
    
    blocks_gdf['GEOID20'] = blocks_gdf['GEOID20'].astype(str)
    target_geoid20 = str(target_geoid20)

    # Filter for the target block
    target_block = blocks_gdf[blocks_gdf['GEOID20'] == target_geoid20]

    if target_block.empty:
        print(f"Block with GEOID20 '{target_geoid20}' not found.")
        return

    # Ensure POP20 and HOUSING20 columns exist
    if 'POP20' not in target_block.columns or 'HOUSING20' not in target_block.columns:
        print("Error: 'POP20' or 'HOUSING20' columns not found for the selected block.")
        return

    # Get population and housing data
    population = target_block['POP20'].iloc[0]
    housing_units = target_block['HOUSING20'].iloc[0]

    # Create plot
    fig, ax = plt.subplots(1, 1, figsize=(12, 12))

    # Plot the target block
    # Use a contrasting color and some transparency to see basemap
    target_block.to_crs(epsg=3857).plot(
        ax=ax,
        alpha=0.5,
        edgecolor='red',
        facecolor='yellow',
        linewidth=2,
        label=f"Block {target_geoid20}"
    )

    # Add basemap from OpenStreetMap (often shows buildings)
    # Adjust zoom level if needed, or let contextily auto-determine
    try:
        ctx.add_basemap(
            ax,
            crs=target_block.to_crs(epsg=3857).crs.to_string(),
            source=ctx.providers.OpenStreetMap.Mapnik, # This source usually has building footprints
            zoom='auto' # Let contextily determine zoom from bounds, or set manually e.g. 18
        )
    except Exception as e:
        print(f"Could not add basemap: {e}")


    # Add text annotation for population and housing
    # Get centroid of the block for placing the text
    # Ensure the geometry is not empty and is valid
    if not target_block.geometry.iloc[0].is_empty and target_block.geometry.iloc[0].is_valid:
        centroid = target_block.geometry.iloc[0].centroid
        
        # Project centroid to the plot's CRS (EPSG:3857) for correct annotation placement
        centroid_map_crs = gpd.GeoSeries([centroid], crs=target_block.crs).to_crs(epsg=3857).iloc[0]

        ax.annotate(
            text=f"GEOID: {target_geoid20}\nPopulation (POP20): {population}\nHousing Units (HOUSING20): {housing_units}",
            xy=(centroid_map_crs.x, centroid_map_crs.y),
            xytext=(5, 5),  # Offset text slightly
            textcoords="offset points",
            bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="black", alpha=0.7),
            fontsize=9
        )
    else:
        print(f"Warning: Geometry for block {target_geoid20} is empty or invalid. Cannot place annotation.")

    ax.set_title(title)
    ax.set_axis_off()
    plt.legend()
    plt.show()

plot_block_with_population_housing_and_buildings(
    blocks_gdf=census_data['blocks'],
    target_geoid20=random.sample(census_data['blocks']['GEOID20'].tolist(), 1)[0],
    title=f"Details for Block"
)

## 2. NREL Data Analysis

This section evaluates the NREL building characteristics data from Step 2, including:
- Building vintage distributions
- Energy characteristics
- Spatial coverage within the target region

**Key Outputs to Validate:**
- NREL parquet file with building data
- Vintage distribution statistics
- Spatial alignment with census boundaries

In [ ]:
def analyze_nrel_data(nrel_path) -> None:
    """Perform comprehensive analysis of NREL data."""
    
    # Check for any .parquet files in the NREL directory and select first and read to dataframe: 
    parquet_files = list(nrel_path.glob('*.parquet'))
    if not parquet_files:
        print("No .parquet files found in the NREL directory")
        return
    
    data = pd.read_parquet(parquet_files[0])
    
    print("=" * 60)
    print("NREL DATA ANALYSIS")
    print("=" * 60)
    
    # Basic statistics
    print(f"\n📊 BASIC STATISTICS:")
    print(f"  - Total NREL records: {len(data):,}")
    print(f"  - Columns available: {len(data.columns)}")
    
    # Coverage check
    if len(data) > 100:
        print(f"  ✅ Dataset has good coverage (>100 records)")
    else:
        print(f"  ⚠️ Small dataset - (<100 records)")
    
    # Vintage Distribution Analysis
    print(f"\n🏠 VINTAGE DISTRIBUTION ANALYSIS:")
    
    if 'in.vintage' in data.columns:
        vintage_counts = data['in.vintage'].value_counts().sort_index()
        total_records = len(data)
        
        print(f"  📈 Vintage Distribution:")
        print(f"     {'Vintage':<12} {'Count':<8} {'Percentage':<10}")
        print(f"     {'-'*32}")
        
        for vintage, count in vintage_counts.items():
            percentage = (count / total_records) * 100
            print(f"     {vintage:<12} {count:<8} {percentage:>8.2f}%")
        
        # Create vintage distribution visualization - pie chart only
        fig, ax = plt.subplots(1, 1, figsize=(8, 8))
        vintage_counts.plot(kind='pie', ax=ax, autopct='%1.1f%%', startangle=90)
        ax.set_title('NREL Vintage Distribution')
        ax.set_ylabel('')
        
        plt.tight_layout()
        plt.show()
    else:
        print(f"  ❌ 'in.vintage' column not found")
    
    # Building Type Distribution Analysis
    print(f"\n🏢 BUILDING TYPE DISTRIBUTION ANALYSIS:")
    
    building_type_col = 'in.geometry_building_type_acs'
    if building_type_col in data.columns:
        
        def map_building_type(bt: str) -> str:
            """Map NREL building types to simplified codes."""
            if pd.isna(bt):
                return "Unknown"
            
            bt_str = str(bt).strip()
            
            if bt_str == "Single-Family Detached":
                return "SFH"
            elif bt_str == "Mobile Home":
                return "SFH"
            elif bt_str == "Single-Family Attached":
                return "TH"
            elif bt_str in ["2 Unit", "3 or 4 Unit", "5 to 9 Unit"]:
                return "MFH"
            elif bt_str in ["10 to 19 Unit", "20 to 49 Unit", "50 or more Unit"]:
                return "AB"
            else:
                return "Other"
        
        # Raw building types
        raw_building_types = data[building_type_col].value_counts()
        total_records = len(data)
        
        print(f"  📈 Raw Building Type Distribution:")
        print(f"     {'Building Type':<25} {'Count':<8} {'Percentage':<10}")
        print(f"     {'-'*45}")
        
        for btype, count in raw_building_types.items():
            percentage = (count / total_records) * 100
            print(f"     {str(btype):<25} {count:<8} {percentage:>8.2f}%")
        
        # Simplified building types
        data_temp = data.copy()
        data_temp['simplified_type'] = data_temp[building_type_col].apply(map_building_type)
        simplified_types = data_temp['simplified_type'].value_counts()
        
        print(f"\n  📊 Simplified Building Type Distribution:")
        print(f"     {'Type':<8} {'Description':<20} {'Count':<8} {'Percentage':<10}")
        print(f"     {'-'*50}")
        
        type_descriptions = {
            'SFH': 'Single Family Home',
            'TH': 'Townhouse',
            'MFH': 'Multi-Family (2-9)',
            'AB': 'Apartment (10+)',
            'Other': 'Other/Unknown'
        }
        
        for stype, count in simplified_types.items():
            percentage = (count / total_records) * 100
            desc = type_descriptions.get(stype, 'Unknown')
            print(f"     {stype:<8} {desc:<20} {count:<8} {percentage:>8.2f}%")
        
        # Building type visualization - pie charts only
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Raw types pie chart
        raw_building_types.plot(kind='pie', ax=axes[0], autopct='%1.1f%%', startangle=90)
        axes[0].set_title('Raw Building Types')
        axes[0].set_ylabel('')
        
        # Simplified types pie chart
        simplified_types.plot(kind='pie', ax=axes[1], autopct='%1.1f%%', startangle=90)
        axes[1].set_title('Simplified Building Types')
        axes[1].set_ylabel('')
        
        plt.tight_layout()
        plt.show()
        
    else:
        print(f"  ❌ '{building_type_col}' column not found")
    
    
    # Check for duplicates
    duplicates = data.duplicated().sum()
    print(f"  - Duplicate records: {'✓' if duplicates == 0 else '⚠️'} ({duplicates} duplicates)")
    

nrel_path = Path(REGION_PATH) / "NREL"
analyze_nrel_data(nrel_path)

## 3. OSM Data Analysis

This section evaluates the OpenStreetMap data extraction from Step 3, including:
- Infrastructure features (buildings, pois, landuse, powerfeatures)

In [ ]:
def load_and_validate_osm_data(region_path: Path) -> dict:
    """Load and validate OSM data outputs."""
    osm_path = region_path / "OSM"
    results = {}
    
    try:
        # Look for common OSM output files
        osm_files = list(osm_path.glob("*.geojson")) + list(osm_path.glob("*.gpkg"))
        
        if not osm_files:
            print("✗ No OSM output files found")
            return {}
        
        results['files'] = osm_files
        results['data'] = {}
        
        # Load each OSM file
        for file_path in osm_files:
            try:
                data = gpd.read_file(file_path)
                file_key = file_path.stem
                results['data'][file_key] = data
                print(f"✓ Loaded {file_key}: {len(data):,} features")
            except Exception as e:
                print(f"✗ Error loading {file_path.name}: {e}")
        
    except Exception as e:
        print(f"✗ Error accessing OSM data: {e}")
        return {}
    
    return results

def analyze_osm_data(osm_data: dict, census_data: dict) -> None:
    """Perform comprehensive analysis of OSM data."""
    
    if not osm_data or not osm_data.get('data'):
        print("No OSM data to analyze")
        return
    
    print("=" * 60)
    print("OSM DATA ANALYSIS")
    print("=" * 60)
    
    total_features = sum(len(gdf) for gdf in osm_data['data'].values())
    
    # Basic statistics
    print(f"\n📊 BASIC STATISTICS:")
    print(f"  - Total OSM datasets: {len(osm_data['data'])}")
    print(f"  - Total features: {total_features:,}")
    
    
    # Geometry type analysis
    print(f"\n🔺 GEOMETRY ANALYSIS:")
    for name, gdf in osm_data['data'].items():
        if len(gdf) > 0:
            geom_types = gdf.geometry.geom_type.value_counts()
            print(f"  - {name}: {geom_types.to_dict()}")
    
    # Visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Feature count by dataset
    dataset_counts = {name: len(gdf) for name, gdf in osm_data['data'].items()}
    axes[0,0].bar(dataset_counts.keys(), dataset_counts.values())
    axes[0,0].set_title('OSM Features by Dataset')
    axes[0,0].set_ylabel('Feature Count')
    axes[0,0].tick_params(axis='x', rotation=45)
    
    # Geometry type distribution
    all_geom_types = {}
    for gdf in osm_data['data'].values():
        if len(gdf) > 0:
            for geom_type, count in gdf.geometry.geom_type.value_counts().items():
                all_geom_types[geom_type] = all_geom_types.get(geom_type, 0) + count
    
    if all_geom_types:
        axes[0,1].pie(all_geom_types.values(), labels=all_geom_types.keys(), autopct='%1.1f%%')
        axes[0,1].set_title('Geometry Type Distribution')
    
    # Spatial plot (if boundary available)
    if census_data and 'boundary' in census_data:
        boundary = census_data['boundary']
        boundary.plot(ax=axes[1,0], color='red', alpha=0.3, edgecolor='red', linewidth=2)
        
        # Plot a sample of OSM features
        colors = plt.cm.Set3(np.linspace(0, 1, len(osm_data['data'])))
        for (name, gdf), color in zip(osm_data['data'].items(), colors):
            if len(gdf) > 0:
                sample_size = min(1000, len(gdf))  # Sample for performance
                gdf.sample(sample_size).plot(ax=axes[1,0], color=color, alpha=0.6, markersize=1, label=name)
        
        axes[1,0].set_title('OSM Features Spatial Distribution')
        axes[1,0].legend()
    
    # Missing values analysis
    all_missing = {}
    for name, gdf in osm_data['data'].items():
        if len(gdf) > 0:
            missing = gdf.isnull().sum().sum()
            all_missing[name] = missing
    
    if all_missing:
        axes[1,1].bar(all_missing.keys(), all_missing.values())
        axes[1,1].set_title('Missing Values by Dataset')
        axes[1,1].set_ylabel('Missing Value Count')
        axes[1,1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()

# Execute OSM data analysis
osm_data = load_and_validate_osm_data(REGION_PATH)
analyze_osm_data(osm_data, census_data)

In [ ]:
# Enhanced OSM Data Analysis with ydata_profiling
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np
from shapely.geometry import Point, Polygon
import warnings
import contextily as ctx
from ydata_profiling import ProfileReport
import folium
from folium import plugins
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

def load_and_validate_osm_data(region_path: Path) -> dict:
    """Load and validate OSM data outputs."""
    osm_path = region_path / "OSM"
    results = {}
    
    try:
        # Look for common OSM output files
        osm_files = list(osm_path.glob("*.geojson")) + list(osm_path.glob("*.gpkg"))
        
        if not osm_files:
            print("✗ No OSM output files found")
            return {}
        
        results['files'] = osm_files
        results['data'] = {}
        
        # Load each OSM file
        for file_path in osm_files:
            try:
                data = gpd.read_file(file_path)
                file_key = file_path.stem
                results['data'][file_key] = data
                print(f"✓ Loaded {file_key}: {len(data):,} features")
            except Exception as e:
                print(f"✗ Error loading {file_path.name}: {e}")
        
    except Exception as e:
        print(f"✗ Error accessing OSM data: {e}")
        return {}
    
    return results

def analyze_osm_data(osm_data: dict, census_data=None):
    """
    Comprehensive analysis of OSM datasets: power, buildings, pois, landuse
    """
    
    if not osm_data or 'data' not in osm_data:
        print("✗ No OSM data available for analysis")
        return
    
    # Extract datasets from osm_data structure
    datasets = osm_data['data']
    
    # Create output directory for reports
    if osm_data.get('files'):
        base_path = osm_data['files'][0].parent
        reports_dir = base_path / "analysis_reports"
    else:
        reports_dir = Path("./analysis_reports")
    
    reports_dir.mkdir(exist_ok=True)
    
    # Map dataset keys to standard names
    dataset_mapping = {}
    for key, gdf in datasets.items():
        key_lower = key.lower()
        if 'power' in key_lower:
            dataset_mapping['power'] = gdf
        elif 'building' in key_lower:
            dataset_mapping['buildings'] = gdf
        elif 'poi' in key_lower or 'amenity' in key_lower:
            dataset_mapping['pois'] = gdf
        elif 'landuse' in key_lower or 'land_use' in key_lower:
            dataset_mapping['landuse'] = gdf
    
    # Use mapped datasets
    datasets = dataset_mapping
    
    # 1. POWER ANALYSIS
    if 'power' in datasets:
        print("\n" + "="*50)
        print("POWER INFRASTRUCTURE ANALYSIS")
        print("="*50)
        
        power_gdf = datasets['power']
        
        # Overall statistics
        print(f"\nOverall Power Infrastructure Count: {len(power_gdf):,}")
        
        # Analyze power types
        if 'power' in power_gdf.columns:
            power_types = power_gdf['power'].value_counts()
            print(f"\nPower Type Distribution:")
            for ptype, count in power_types.items():
                print(f"  {ptype}: {count:,} ({count/len(power_gdf)*100:.1f}%)")
        else:
            print("\nNo 'power' column found for type classification")
            power_types = pd.Series({'unknown': len(power_gdf)})
        
        # Create visualizations for power
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        fig.suptitle('Power Infrastructure Analysis', fontsize=16, fontweight='bold')
        
        # Power type distribution bar chart
        axes[0,0].bar(power_types.index, power_types.values, color='skyblue', edgecolor='navy')
        axes[0,0].set_title('Power Infrastructure Count by Type')
        axes[0,0].set_xlabel('Power Type')
        axes[0,0].set_ylabel('Count')
        axes[0,0].tick_params(axis='x', rotation=45)
        
        # Power type distribution pie chart
        axes[0,1].pie(power_types.values, labels=power_types.index, autopct='%1.1f%%', startangle=90)
        axes[0,1].set_title('Power Infrastructure Distribution')
        
        # Geographic distribution of power infrastructure
        power_gdf.plot(ax=axes[1,0], color='red', markersize=20, alpha=0.7)
        axes[1,0].set_title('Geographic Distribution of Power Infrastructure')
        axes[1,0].set_xlabel('Longitude')
        axes[1,0].set_ylabel('Latitude')
        axes[1,0].grid(True, alpha=0.3)
        
        # Substation analysis if available
        if 'substation' in power_gdf.columns:
            substation_types = power_gdf['substation'].value_counts().dropna()
            if len(substation_types) > 0:
                axes[1,1].bar(substation_types.index, substation_types.values, color='orange', edgecolor='darkred')
                axes[1,1].set_title('Substation Type Distribution')
                axes[1,1].set_xlabel('Substation Type')
                axes[1,1].set_ylabel('Count')
                axes[1,1].tick_params(axis='x', rotation=45)
                
                print(f"\nSubstation Type Distribution:")
                for stype, count in substation_types.items():
                    print(f"  {stype}: {count:,}")
            else:
                axes[1,1].text(0.5, 0.5, 'No substation types\nwith data', 
                              ha='center', va='center', transform=axes[1,1].transAxes)
                axes[1,1].set_title('Substation Classification (Empty)')
        else:
            axes[1,1].text(0.5, 0.5, 'No substation\nclassification available', 
                          ha='center', va='center', transform=axes[1,1].transAxes)
            axes[1,1].set_title('Substation Classification (N/A)')
        
        plt.tight_layout()
        plt.savefig(reports_dir / 'power_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Generate ydata_profiling report for power
        power_df = power_gdf.drop(columns=['geometry']).copy()
        power_profile = ProfileReport(power_df, title="Power Infrastructure Profile Report", explorative=True)
        power_profile.to_file(reports_dir / "power_profile_report.html")
        print(f"✓ Power profile report saved to: {reports_dir / 'power_profile_report.html'}")
    

        
    # 2. BUILDINGS ANALYSIS
    if 'buildings' in datasets:
        print("\n" + "="*50)
        print("BUILDINGS ANALYSIS")
        print("="*50)
        
        buildings_gdf = datasets['buildings']
        print(f"\nTotal Buildings Count: {len(buildings_gdf):,}")
        
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        fig.suptitle('Buildings Analysis', fontsize=16, fontweight='bold')
        
        # Building count
        axes[0].bar(['Total Buildings'], [len(buildings_gdf)], color='lightblue', edgecolor='navy')
        axes[0].set_title('Total Building Count')
        axes[0].set_ylabel('Count')
        for i, v in enumerate([len(buildings_gdf)]):
            axes[0].text(i, v + v*0.01, f'{v:,}', ha='center', va='bottom', fontweight='bold')
        
        # Building types analysis
        if 'building' in buildings_gdf.columns:
            building_types = buildings_gdf['building'].value_counts().head(10)
            axes[1].barh(building_types.index, building_types.values, color='lightgreen', edgecolor='darkgreen')
            axes[1].set_title('Top 10 Building Types')
            axes[1].set_xlabel('Count')
            
            print(f"\nBuilding Type Distribution (Top 10):")
            for btype, count in building_types.items():
                print(f"  {btype}: {count:,}")
        else:
            axes[1].text(0.5, 0.5, 'No building type\ninformation available', 
                        ha='center', va='center', transform=axes[1].transAxes)
            axes[1].set_title('Building Types (N/A)')
        
        plt.tight_layout()
        plt.savefig(reports_dir / 'buildings_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
                
        # Generate ydata_profiling report for buildings
        buildings_df = buildings_gdf.drop(columns=['geometry']).copy()
        buildings_profile = ProfileReport(buildings_df, title="Buildings Profile Report", explorative=True)
        buildings_profile.to_file(reports_dir / "buildings_profile_report.html")
        print(f"✓ Buildings profile report saved to: {reports_dir / 'buildings_profile_report.html'}")
    
    if 'pois' in datasets:
        print("\n" + "="*50)
        print("POINTS OF INTEREST ANALYSIS")
        print("="*50)
        
        pois_gdf = datasets['pois']
        print(f"\nTotal POIs Count: {len(pois_gdf):,}")
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        fig.suptitle('Points of Interest Analysis', fontsize=16, fontweight='bold')
        
        # POI count
        axes[0].bar(['Total POIs'], [len(pois_gdf)], color='lightcoral', edgecolor='darkred')
        axes[0].set_title('Total POI Count')
        axes[0].set_ylabel('Count')
        for i, v in enumerate([len(pois_gdf)]):
            axes[0].text(i, v + v*0.01, f'{v:,}', ha='center', va='bottom', fontweight='bold')
        
        # POI amenity types
        if 'amenity' in pois_gdf.columns:
            amenity_types = pois_gdf['amenity'].value_counts().head(10)
            axes[1].barh(amenity_types.index, amenity_types.values, color='lightpink', edgecolor='darkred')
            axes[1].set_title('Top 10 Amenity Types')
            axes[1].set_xlabel('Count')
            
            print(f"\nAmenity Type Distribution (Top 10):")
            for atype, count in amenity_types.items():
                print(f"  {atype}: {count:,}")
        else:
            axes[1].text(0.5, 0.5, 'No amenity type\ninformation available', 
                        ha='center', va='center', transform=axes[1].transAxes)
            axes[1].set_title('Amenity Types (N/A)')
        
        # Geographic distribution
        pois_sample = pois_gdf.sample(min(1000, len(pois_gdf))) if len(pois_gdf) > 1000 else pois_gdf
        pois_sample.plot(ax=axes[2], color='purple', alpha=0.6, markersize=8)
        axes[2].set_title('Geographic Distribution of POIs')
        axes[2].set_xlabel('Longitude')
        axes[2].set_ylabel('Latitude')
        axes[2].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(reports_dir / 'pois_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Generate ydata_profiling report for POIs
        pois_df = pois_gdf.drop(columns=['geometry']).copy()
        pois_profile = ProfileReport(pois_df, title="POIs Profile Report", explorative=True)
        pois_profile.to_file(reports_dir / "pois_profile_report.html")
        print(f"✓ POIs profile report saved to: {reports_dir / 'pois_profile_report.html'}")
    
    # 4. LANDUSE ANALYSIS
    if 'landuse' in datasets:
        print("\n" + "="*50)
        print("LANDUSE ANALYSIS")
        print("="*50)
        
        landuse_gdf = datasets['landuse']
        print(f"\nTotal Landuse Areas Count: {len(landuse_gdf):,}")
        
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))
        fig.suptitle('Landuse Analysis', fontsize=16, fontweight='bold')
        
        # Landuse count
        axes[0].bar(['Total Landuse Areas'], [len(landuse_gdf)], color='lightgreen', edgecolor='darkgreen')
        axes[0].set_title('Total Landuse Count')
        axes[0].set_ylabel('Count')
        for i, v in enumerate([len(landuse_gdf)]):
            axes[0].text(i, v + v*0.01, f'{v:,}', ha='center', va='bottom', fontweight='bold')
        
        # Landuse types
        if 'landuse' in landuse_gdf.columns:
            landuse_types = landuse_gdf['landuse'].value_counts().head(10)
            axes[1].barh(landuse_types.index, landuse_types.values, color='lightblue', edgecolor='darkblue')
            axes[1].set_title('Top 10 Landuse Types')
            axes[1].set_xlabel('Count')
            
            print(f"\nLanduse Type Distribution (Top 10):")
            for ltype, count in landuse_types.items():
                print(f"  {ltype}: {count:,}")
        else:
            axes[1].text(0.5, 0.5, 'No landuse type\ninformation available', 
                          ha='center', va='center', transform=axes[0,1].transAxes)
            axes[1].set_title('Landuse Types (N/A)')
        
        # Geographic distribution
        landuse_sample = landuse_gdf.sample(min(500, len(landuse_gdf))) if len(landuse_gdf) > 500 else landuse_gdf
        landuse_sample.plot(ax=axes[2], color='green', alpha=0.5, edgecolor='darkgreen')
        axes[2].set_title('Geographic Distribution of Landuse')
        axes[2].set_xlabel('Longitude')
        axes[2].set_ylabel('Latitude')
        axes[2].grid(True, alpha=0.3)
        
        
        plt.tight_layout()
        plt.savefig(reports_dir / 'landuse_analysis.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Generate ydata_profiling report for landuse
        landuse_df = landuse_gdf.drop(columns=['geometry']).copy()
        landuse_profile = ProfileReport(landuse_df, title="Landuse Profile Report", explorative=True)
        landuse_profile.to_file(reports_dir / "landuse_profile_report.html")
        print(f"✓ Landuse profile report saved to: {reports_dir / 'landuse_profile_report.html'}")
    
    # 5. COMBINED OVERVIEW
    print("\n" + "="*50)
    print("COMBINED OVERVIEW")
    print("="*50)
    
    # Create summary plot
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    
    dataset_counts = {name: len(gdf) for name, gdf in datasets.items()}
    colors = ['red', 'blue', 'purple', 'green']
    
    if dataset_counts:
        bars = ax.bar(dataset_counts.keys(), dataset_counts.values(), 
                      color=colors[:len(dataset_counts)], alpha=0.7, edgecolor='black')
        
        # Add count labels on bars
        for bar, count in zip(bars, dataset_counts.values()):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
                    f'{count:,}', ha='center', va='bottom', fontweight='bold')
        
        ax.set_title('OSM Dataset Feature Counts Comparison', fontsize=16, fontweight='bold')
        ax.set_ylabel('Number of Features')
        ax.set_xlabel('Dataset Type')
        ax.set_yscale('log')  # Log scale for better comparison
        
        plt.tight_layout()
        plt.savefig(reports_dir / 'combined_overview.png', dpi=300, bbox_inches='tight')
        plt.show()
        
    else:
        print("No datasets found for analysis")
    
    print(f"\n✓ All analysis reports saved to: {reports_dir}")
    return datasets, reports_dir

# Execute OSM data analysis
osm_data = load_and_validate_osm_data(REGION_PATH)
datasets, reports_dir = analyze_osm_data(osm_data, census_data)

## 4. Building Data Analysis

This section evaluates the building data processing from Steps 3.5 and 4, including:
- Building classification heuristics

**Key Outputs to Validate:**
- Building classification results

In [ ]:
def load_building_shapefiles(region_path: Path) -> dict:
    """Load residential and non-residential building shapefiles from SHP folder."""
    shp_path = region_path / "BUILDINGS_OUTPUT" / "SHP"
    results = {}
    
    try:
        # Look for residential and non-residential shapefiles
        print(shp_path)

        shp_files = list(shp_path.glob("*.shp"))
        print(shp_files)
        
        for shp_file in shp_files:
            filename = shp_file.stem.lower()
            
            if 'residential' in filename and 'non' not in filename:
                try:
                    results['residential'] = gpd.read_file(shp_file)
                    print(f"✓ Loaded residential buildings: {len(results['residential']):,} buildings")
                except Exception as e:
                    print(f"✗ Error loading residential shapefile: {e}")
                    
            elif 'non_residential' in filename or 'nonresidential' in filename:
                try:
                    results['non_residential'] = gpd.read_file(shp_file)
                    print(f"✓ Loaded non-residential buildings: {len(results['non_residential']):,} buildings")
                except Exception as e:
                    print(f"✗ Error loading non-residential shapefile: {e}")
        
        if not results:
            print("✗ No building shapefiles found in SHP folder")
            
    except Exception as e:
        print(f"✗ Error accessing SHP folder: {e}")
    
    return results

def analyze_building_distributions(buildings_gdf: gpd.GeoDataFrame, building_type: str, columns_to_analyze: list) -> None:
    """
    Reusable function to analyze building feature distributions.
    
    Args:
        buildings_gdf: GeoDataFrame with building data
        building_type: Type description (e.g., "Residential", "Non-Residential")
        columns_to_analyze: List of column names to analyze
    """
    if buildings_gdf is None or len(buildings_gdf) == 0:
        print(f"No {building_type.lower()} building data to analyze")
        return
    
    print("=" * 60)
    print(f"{building_type.upper()} BUILDINGS DISTRIBUTION ANALYSIS")
    print("=" * 60)
    
    # Basic statistics
    print(f"\n📊 BASIC STATISTICS:")
    print(f"  - Total {building_type.lower()} buildings: {len(buildings_gdf):,}")
    print(f"  - Available columns: {list(buildings_gdf.columns)}")
    
    # Check which columns are available
    available_columns = [col for col in columns_to_analyze if col in buildings_gdf.columns]
    missing_columns = [col for col in columns_to_analyze if col not in buildings_gdf.columns]
    
    if missing_columns:
        print(f"  ⚠️ Missing columns: {missing_columns}")
    
    if not available_columns:
        print(f"  ❌ None of the target columns found in {building_type.lower()} data")
        return
    
    print(f"  ✓ Analyzing columns: {available_columns}")
    
    # Create subplots for distributions
    n_cols = len(available_columns)
    if n_cols == 0:
        return
    
    # Calculate subplot layout
    n_rows = (n_cols + 2) // 3  # Max 3 columns per row
    n_subplot_cols = min(3, n_cols)
    
    fig, axes = plt.subplots(n_rows, n_subplot_cols, figsize=(5*n_subplot_cols, 4*n_rows))
    
    # Handle single subplot case
    if n_cols == 1:
        axes = [axes]
    elif n_rows == 1:
        axes = axes if n_cols > 1 else [axes]
    else:
        axes = axes.flatten()
    
    # Analyze each column
    for i, column in enumerate(available_columns):
        ax = axes[i]
        data = buildings_gdf[column].dropna()
        
        print(f"\n📈 {column.upper()} ANALYSIS:")
        print(f"  - Total valid values: {len(data):,}")
        print(f"  - Missing values: {buildings_gdf[column].isna().sum():,}")
        
        if len(data) == 0:
            ax.text(0.5, 0.5, f'No data for\n{column}', ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{column}')
            continue
        
        # Handle different data types
        if column == 'floor_area':
            # Bin floor areas for better visualization
            bins = [0, 50, 100, 200, 500, 1000, 2000, 5000, float('inf')]
            labels = ['<50', '50-100', '100-200', '200-500', '500-1k', '1k-2k', '2k-5k', '>5k']
            
            try:
                binned_data = pd.cut(data, bins=bins, labels=labels, include_lowest=True)
                value_counts = binned_data.value_counts().sort_index()
                
                print(f"  - Mean area: {data.mean():.1f} m²")
                print(f"  - Median area: {data.median():.1f} m²")
                print(f"  - Area range: {data.min():.1f} - {data.max():.1f} m²")
                
                # Distribution
                print(f"  - Area distribution:")
                for area_range, count in value_counts.items():
                    pct = (count / len(data)) * 100
                    print(f"    * {area_range} m²: {count:,} ({pct:.1f}%)")
                
                value_counts.plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')
                ax.set_title(f'{column} Distribution (m²)')
                ax.set_xlabel('Area Range (m²)')
                ax.tick_params(axis='x', rotation=45)
                
            except Exception as e:
                print(f"  ⚠️ Error binning floor area: {e}")
                data.hist(bins=20, ax=ax, alpha=0.7, edgecolor='black')
                ax.set_title(f'{column} Distribution')
        
        elif data.dtype in ['object', 'string']:
            # Categorical data
            value_counts = data.value_counts().head(10)  # Top 10 categories
            
            print(f"  - Unique values: {data.nunique()}")
            print(f"  - Distribution:")
            for value, count in value_counts.items():
                pct = (count / len(data)) * 100
                print(f"    * {value}: {count:,} ({pct:.1f}%)")
            
            value_counts.plot(kind='bar', ax=ax, color='lightgreen', edgecolor='black')
            ax.set_title(f'{column} Distribution')
            ax.set_xlabel(column)
            ax.tick_params(axis='x', rotation=45)
        
        else:
            # Numeric data
            try:
                print(f"  - Mean: {data.mean():.2f}")
                print(f"  - Median: {data.median():.2f}")
                print(f"  - Range: {data.min()} - {data.max()}")
                
                # Show value counts if reasonable number of unique values
                if data.nunique() <= 20:
                    value_counts = data.value_counts().sort_index()
                    print(f"  - Distribution:")
                    for value, count in value_counts.items():
                        pct = (count / len(data)) * 100
                        print(f"    * {value}: {count:,} ({pct:.1f}%)")
                    
                    value_counts.plot(kind='bar', ax=ax, color='orange', edgecolor='black')
                    ax.set_title(f'{column} Distribution')
                    ax.set_xlabel(column)
                else:
                    # Too many unique values, use histogram
                    data.hist(bins=20, ax=ax, alpha=0.7, edgecolor='black')
                    ax.set_title(f'{column} Distribution')
                    ax.set_xlabel(column)
                    
            except Exception as e:
                print(f"  ⚠️ Error analyzing numeric data: {e}")
                data.hist(bins=20, ax=ax, alpha=0.7, edgecolor='black')
                ax.set_title(f'{column} Distribution')
        
        ax.set_ylabel('Count')
    
    # Hide empty subplots
    for j in range(len(available_columns), len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    plt.show()

def analyze_final_building_classifications(region_path: Path) -> None:
    """Main function to analyze final building classification outputs."""
    
    print("🏗️ FINAL BUILDING CLASSIFICATION ANALYSIS")
    print("=" * 80)
    
    # Load building shapefiles
    building_data = load_building_shapefiles(region_path)
    
    if not building_data:
        print("No building shapefiles found for analysis")
        return
    
    # Define columns to analyze for each building type
    residential_columns = [
        'floor_area', 'building_u', 'free_walls', 
        'building_t', 'occupants', 'floors', 'constructi'
    ]
    
    non_residential_columns = [
        'floor_area', 'building_use', 'free_walls'
    ]
    
    # Analyze residential buildings
    if 'residential' in building_data:
        analyze_building_distributions(
            building_data['residential'], 
            "Residential", 
            residential_columns
        )
    
    # Analyze non-residential buildings
    if 'non_residential' in building_data:
        analyze_building_distributions(
            building_data['non_residential'], 
            "Non-Residential", 
            non_residential_columns
        )
    
    # Summary comparison
    if 'residential' in building_data and 'non_residential' in building_data:
        print("\n" + "=" * 60)
        print("BUILDING CLASSIFICATION SUMMARY")
        print("=" * 60)
        
        res_count = len(building_data['residential'])
        non_res_count = len(building_data['non_residential'])
        total_count = res_count + non_res_count
        
        print(f"\n📊 OVERALL STATISTICS:")
        print(f"  - Total buildings classified: {total_count:,}")
        print(f"  - Residential buildings: {res_count:,} ({(res_count/total_count)*100:.1f}%)")
        print(f"  - Non-residential buildings: {non_res_count:,} ({(non_res_count/total_count)*100:.1f}%)")
        
        # Create summary comparison chart
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        
        categories = ['Residential', 'Non-Residential']
        counts = [res_count, non_res_count]
        colors = ['lightblue', 'lightcoral']
        
        bars = ax.bar(categories, counts, color=colors, edgecolor='black')
        ax.set_title('Building Classification Summary')
        ax.set_ylabel('Number of Buildings')
        
        # Add count labels on bars
        for bar, count in zip(bars, counts):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + total_count*0.01,
                   f'{count:,}', ha='center', va='bottom', fontweight='bold')
        
        plt.tight_layout()
        plt.show()

# Execute the analysis
analyze_final_building_classifications(REGION_PATH)

## 5. Road Network Analysis TODO

This section evaluates the road network generation from Step 5, including:
- Network topology and connectivity
- Routable network validation
- Spatial coverage and completeness

**Key Outputs to Validate:**
- Generated road network file (GPKG/GeoJSON)
- Network connectivity and routing capability
- Integration with regional boundaries